# Load libraries & helper functions

In [1]:
%matplotlib inline
from helpers import *
# Turn interactive plotting off
plt.ioff()

# Declare parameters

In [ ]:
config.DATADIR      = '/scratch/duboisjx/data/CamCAN/MRI'
config.fmriRun      = 'Movie'
# use volume or surface data
config.isCifti      = False
# use ICA-FIX input
config.isDataClean  = False

config.overwrite    = False

config.pipelineName = 'Finn'
config.preWhitening = False
config.Operations= [
    ['VoxelNormalization',      1, ['zscore']],
    ['Detrending',              2, ['legendre', 3, 'WMCSF']],
    ['TissueRegression',        3, ['WMCSF', 'GM']],
    ['MotionRegression',        4, ['R dR']],
    ['TemporalFiltering',       5, ['Gaussian', 1]],
    ['Detrending',              6, ['legendre', 3,'GM']],
    ['GlobalSignalRegression',  7, ['GS']],
]

# submit jobs with sge
config.queue        = True
# make sure to set memory requirements according to data size
# 15G for HCP data!
config.maxvmem      = '4G'

# parcellation for FC matrix
config.parcellationName = 'shen2013'
config.parcellationFile = '/scratch/duboisjx/data/parcellations/shenetal_neuroimage2013_new/shen_3mm_268_parcellation.nii.gz'
config.nParcels         = 268

subjects = [d for d in listdir(config.DATADIR) if d[0:2]=="CC"]
subjects.sort()
print 'Found {} subjects'.format(len(subjects))
#print subjects

Found 652 subjects


# Do work

### preprocess everybody

In [ ]:
keep      = np.zeros((len(subjects)),dtype=np.bool_)
i=0
displayPlot=False
for config.subject in subjects:
    if not config.queue:
        print 'SUB {}/{}: {}'.format(i+1,len(subjects),config.subject)
    keep[i] = runPipelinePar()
    i = i + 1
        
print 'Keeping {}/{} subjects'.format(np.sum(keep),len(subjects))

if config.queue:
    if len(config.joblist) != 0:
        while True:
            nleft = len(config.joblist)
            for i in range(nleft):
                myCmd = "qstat | grep ' {} '".format(config.joblist[i])
                isEmpty = False
                try:
                    cmdOut = check_output(myCmd, shell=True)
                except CalledProcessError as e:
                    isEmpty = True
                finally:
                    if isEmpty:
                        nleft = nleft-1
            if nleft == 0:
                break
            else:
                print 'Waiting for {} subjects to complete...'.format(nleft)
            sleep(10)
    print 'All done!!'

CC110062 missing
CC120376 missing
CC120409 missing
CC120462 missing
CC120470 missing
CC120640 missing
CC120987 missing
CC410129 missing
CC410390 missing
CC420061 missing
CC420091 missing
CC420137 missing
CC420197 missing


### look at QC

In [ ]:
fdScores  = np.zeros((len(subjects)),dtype=np.float32)
fcMats    = np.zeros((config.nParcels,config.nParcels,len(subjects)),dtype=np.float32)
fcMats_dn = np.zeros((config.nParcels,config.nParcels,len(subjects)),dtype=np.float32)
displayPlot=False
i = 0
config.queue=False
for config.subject in subjects:
    if keep[i]:
        print 'SUB {}/{}: {}'.format(i+1,len(subjects),config.subject)
        runPipelinePar()
        fcMats[:,:,i],fcMats_dn[:,:,i] = plotFC(displayPlot)
        fdScores[i]=np.mean(computeFD())
    i = i + 1

fdScores  = fdScores[keep]
fcMats    = fcMats[:,:,keep]
fcMats_dn = fcMats_dn[:,:,keep]

# rsFC/mean FD plot
# since this is a correlation across subjects, it makes most sense if we have enough subjects
if len(subjects)>20:    
    plotQCrsFC(fcMats,fcMats_dn,fdScores)
    
# deltaR plot
plotDeltaR(fcMats,fcMats_dn)